In [1]:
import pandas as pd
import geopandas as gpd
import os
import requests
import yaml
from sklearn.impute import SimpleImputer

Aggrégation des données

In [2]:
# folder = "../raw_data/year_data/"
# dff = []
# files = [file for file in os.listdir(folder) if file.endswith('.csv')]
# for file in files:
#     file_path = os.path.join(folder, file)
#     if file in ['caracteristiques_2018.csv', 'caracteristiques_2008.csv',
#                 'caracteristiques_2011.csv','caracteristiques_2010.csv',
#                 'caracteristiques_2005.csv','caracteristiques_2006.csv',
#                 'caracteristiques_2007.csv',
#                 'caracteristiques_2013.csv','caracteristiques_2012.csv',
#                 'caracteristiques_2014.csv','caracteristiques_2015.csv',
#                 'caracteristiques_2016.csv','caracteristiques_2017.csv']:
#         encoding = 'latin-1'
#         sep = ','
#     elif file in ['caracteristiques_2009.csv']:
#         encoding = 'latin-1'
#         sep = '\t'
#     elif file in ['vehicules_2019.csv','vehicules_2020.csv','vehicules_2021.csv','vehicules_2022.csv','lieux_2019.csv','lieux_2020.csv','lieux_2021.csv',
#                   'lieux_2022.csv','caracteristiques_2019.csv',
#                   'caracteristiques_2021.csv','caracteristiques_2022.csv',
#                   'caracteristiques_2020.csv' ]:
#         encoding = 'latin-1'

#         sep = ';'
#     elif file in ['2019.csv', '2020.csv', '2021.csv', '2022.csv']:
#         sep = ';'
#         encoding = 'latin-1'
#     else:
#         sep = ','
#         encoding = 'utf-8'
#     print(file_path)
#     df = pd.read_csv(file_path, sep=sep, encoding=encoding)
#     # df = df.applymap(lambda x: str(x).replace('"', ''))
#     dff.append(df)
#     df_chiant = pd.concat(dff)


API POUR REQUEST & SAVE URL


In [3]:
# ### call api for datasets urls


# def get_datasets_url(url=None):
#     url ='https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/'
#     r = requests.get(url).json()
#     return {el['title']: el['latest'] for el in r['resources'] if el['title'].endswith(".csv") and not el['title'].startswith("vehicules-immatricules") }

# lieux_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("lieux")}
# usagers_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("usagers")}
# car_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("car")}
# vehicule_datasets = {i:j for i,j in get_datasets_url().items() if i.startswith("vehicule")}

# all_urls = [lieux_datasets, usagers_datasets, car_datasets,vehicule_datasets]

# ### download csvs if not already

# for url_dict in all_urls:
#     for path, url in url_dict.items():
#         path = '../raw_data/' + path
#         if not os.path.exists(path):
#             response = requests.get(url)
#             if response.status_code == 200:
#                 with open(path, 'wb') as f:
#                     f.write(response.content)

# ### check datasets

# folder = "../raw_data/"
# os.listdir(folder)

In [4]:
# chemin_fichier_yml = '../config.yml'
# chemin_dossier = '../raw_data/'


# with open (chemin_fichier_yml, 'r') as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)
#     rename_config = config.get('rename')

# for old_name, new_name in rename_config.items():
#     chemin_ancien_fichier = os.path.join(chemin_dossier, old_name)
#     chemin_nouveau_fichier = os.path.join(chemin_dossier, new_name)

#     if os.path.exists(chemin_ancien_fichier):
#         os.rename(chemin_ancien_fichier, chemin_nouveau_fichier)
#         print(f"Fichier renommé : {old_name} -> {new_name}")
#     else:
#         print(f"Fichier non trouvé : {old_name}")


In [5]:


# def concat_files(starting_word):

#     chemin_fichier_yml = '../config.yml'
#     with open(chemin_fichier_yml, 'r') as f:
#         config = yaml.safe_load(f)
#         config_sep = config.get('sep')
#         config_encoding = config.get('encoding')

#     chemin_dossier = '../raw_data/'

#     df_concat = pd.DataFrame()
#     files = [file for file in os.listdir(chemin_dossier) if file.endswith('.csv') and file.startswith(starting_word)]

#     print(files)
#     for file in files:
#         chemin_fichier = os.path.join(chemin_dossier, file)

#         if file in config_sep:
#             sep = config_sep[file]
#         else:
#             sep = ','

#         if file in config_encoding:
#             encoding = config_encoding[file]
#         else:
#             encoding = 'utf-8'

#         df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)

#         df_concat = pd.concat([df_concat, df1])

#     return df_concat


In [6]:
# carac_df = concat_files("caracteristiques")
# lieux_df = concat_files("lieux")
# usager_df = concat_files("usagers")
# vehi_df = concat_files("vehicules")

# vehi_df.to_csv('vehi.csv', index=False)
# carac_df.to_csv('carac.csv', index=False)
# lieux_df.to_csv('lieux.csv', index=False)
# usager_df.to_csv('usagers.csv', index=False)


In [7]:
lieux_df = pd.read_csv('lieux.csv')

print(f'La taille du fichier est {lieux_df.shape}')

/var/folders/jw/qw353j7x7h970k1t74h_j5nc0000gn/T/ipykernel_71618/413021987.py:1: DtypeWarning: Columns (2,6,7,8,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux_df = pd.read_csv('lieux.csv')


La taille du fichier est (1176873, 19)


In [8]:
lieux_df['circ'].value_counts()
lieux_df['circ'] = lieux_df['circ'].replace({0: 2, -1: 2})
lieux_df['circ'].fillna(2, inplace=True)

# on remplace 60k de valeurs nulles par la plus fréquentes

In [9]:
lieux_df['circ'].value_counts()

2.0    805547
1.0    208883
3.0    155773
4.0      6670
Name: circ, dtype: int64

In [10]:
lieux_df['nbv'] = pd.to_numeric(lieux_df['nbv'], errors='coerce')

lieux_df['nbv'] = lieux_df['nbv'].where(lieux_df['nbv'] <= 10, 2)
lieux_df['nbv'] = lieux_df['nbv'].replace({-1: 2})
lieux_df['nbv'].fillna(2, inplace=True)
lieux_df['nbv'].value_counts()

# on remplace les valeurs abbérantes(>10) et les valeurs nulles par la valeur la plus fréquente

2.0     675002
1.0     137502
4.0     115348
0.0     114127
3.0      91434
6.0      22437
5.0      12866
8.0       4889
7.0       1715
10.0      1022
9.0        531
Name: nbv, dtype: int64

In [11]:

lieux_df['vosp'] = lieux_df['vosp'].replace({-1: 0})
lieux_df['vosp'].fillna(0, inplace=True)
lieux_df['vosp'].value_counts()

# on remplace les valeurs nulles (1k) par la plus fréquente

0.0    1094828
3.0      34397
1.0      29795
2.0      17853
Name: vosp, dtype: int64

In [12]:

lieux_df['prof'] = lieux_df['prof'].replace({-1: 1,0: 1})
lieux_df['prof'].fillna(1, inplace=True)

# on remplace les valeurs nulles (65k) par la plus fréquente

In [13]:
lieux_df['pr'].isnull().sum()
# on drop la colonne pr et la pr1 qui sera pas plus utile


475484

In [14]:
lieux_df['plan'].value_counts()
lieux_df['plan'] = lieux_df['plan'].replace({-1: 1,0: 1})
lieux_df['plan'].fillna(1, inplace=True)

# on remplace les valeurs nulles (65k) par la plus fréquente

In [15]:
lieux_df['lartpc'].value_counts()

#30% de valeurs nulles on drop la colonne

0.0      746701
15.0      33446
10.0      19535
20.0      16510
0.0       12131
          ...  
77.0          1
112.0         1
219.0         1
293.0         1
5.5           1
Name: lartpc, Length: 605, dtype: int64

In [16]:
pd.set_option('display.max_rows', None)
print(lieux_df['larrout'].value_counts())

# on drop, trop valeurs aberrantes

0.0                  257948
 -1                  149480
60.0                  68171
70.0                  54081
80.0                  24250
50.0                  22056
120.0                 20867
0.0                   20532
100.0                 18376
140.0                 17045
90.0                  16295
105.0                 14542
30.0                  13555
40.0                  11697
35.0                  10741
65.0                   9878
0                      9475
75.0                   9456
72.0                   7368
62.0                   6590
68.0                   6174
210.0                  6028
55.0                   5943
150.0                  5781
60.0                   5547
64.0                   5423
58.0                   5153
63.0                   4973
70.0                   4797
61.0                   4728
110.0                  4655
71.0                   4489
74.0                   4456
66.0                   4429
73.0                   4308
45.0                

In [17]:
lieux_df['surf'].value_counts()
lieux_df['surf'] = lieux_df['surf'].replace({-1: 1,0: 1, 9: 1})
lieux_df['surf'].fillna(1, inplace=True)

# on remplace les valeurs nulles et non pertinentes (autre) par la plus fréquente

In [18]:
lieux_df['infra'].value_counts()
lieux_df['infra'] = lieux_df['infra'].replace({-1: 1})
lieux_df['infra'].fillna(1, inplace=True)

# on remplace les valeurs nulles par la plus fréquente,
# 0 ne l'étant pas car ça veut dire qu'il n'y a pas d'infra,
# 9 peut être pertinent car c'est une infra 'autre'

In [19]:
lieux_df['situ'].value_counts()
lieux_df['situ'] = lieux_df['situ'].replace({-1: 1, 0: 1})
lieux_df['situ'].fillna(1, inplace=True)
# on remplace les valeurs nulles par la plus fréquentes

In [20]:
lieux_df['vma'].value_counts()
lieux_df['vma'].isnull().sum()

958469

In [37]:
lieux_df['catr'].fillna(4, inplace=True)

In [38]:
lieux_df_clean = lieux_df.drop(columns=['voie', 'v1', 'v2', 'pr', 'pr1', 'lartpc', 'larrout', 'vma', 'env1'])


In [39]:
lieux_df_clean.shape

(1176873, 10)

In [40]:
print(lieux_df_clean[lieux_df_clean.isna().any(axis=1)])

Empty DataFrame
Columns: [Num_Acc, catr, circ, nbv, vosp, prof, plan, surf, infra, situ]
Index: []
